In [1]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
#from numba import jit
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch.cuda import FloatTensor

# Req for package
sys.path.append("../")
from SkinLearning.Utils.NN import train, test, DEVICE


torch.backends.cudnn.benchmark = True

In [7]:
# Folder name will correspond to index of sample
class SkinDataset(Dataset):
    def __init__(self, scaler, signalFolder="D:/SamplingResults2", sampleFile="../Data/newSamples.pkl", runs=range(65535), steps=128):
        # Load both disp1 and disp2 from each folder
        # Folders ordered according to index of sample
        # Use the corresponding sample as y -> append probe?
        self.input = []
        self.output = []
        
        with open(f"{sampleFile}", "rb") as f:
             samples = pickle.load(f)
        
        self.min = np.min(samples[runs])
        self.max = np.max(samples[runs])
        
        
        for run in tqdm(runs):
            inp = []
            fail = False
            
            files = os.listdir(f"{signalFolder}/{run}/")
            
            if files != ['Disp1.csv', 'Disp2.csv']:
                continue
            
            for file in files:
                a = pd.read_csv(f"{signalFolder}/{run}/{file}")
                a.rename(columns = {'0':'x', '0.1': 'y'}, inplace = True)
                
                # Skip if unconverged
                if a['x'].max() != 7.0:
                    fail = True
                    break

                # Interpolate curve for consistent x values
                xNew = np.linspace(0, 7, num=steps, endpoint=False)
                interped = interp1d(a['x'], a['y'], kind='cubic', fill_value="extrapolate")(xNew)
                    
                
                inp.append(interped.astype("float32"))
            
            if not fail:
                if len(inp) != 2:
                    raise Exception("sdf")

                self.input.append(inp)
                self.output.append(samples[int(run)])
        
        scaler.fit(self.output)
        self.output = scaler.fit_transform(self.output)
        self.output = tensor(self.output).type(FloatTensor)
        
        self.input = [waveletExtraction(sample) for sample in self.input]
        self.input = tensor(self.input).type(FloatTensor)
        
        
    def __len__(self):
        return len(self.output)
    
    def __getitem__(self, idx):
        sample = {"input": self.input[idx], "output": self.output[idx]}
        return sample
    
    

In [8]:
"""
    Creates the data set from filtered samples
    Returns the dataset and the scaler
"""
def getDataset(**kwargs):
    # Get filtered data
    if not 'runs' in kwargs.keys():
        with open("../Data/filtered.pkl", "rb") as f:
            runs = pickle.load(f)

        kwargs['runs'] = runs

    scaler = MinMaxScaler()
    dataset = SkinDataset(scaler=scaler, **kwargs)

    return dataset, scaler

In [9]:
"""
    Creates a train/test split from the given data
    Returns train and test data loaders
"""
def getSplit(dataset, p1=0.8):
    train_n = int(p1 * len(dataset))
    test_n = len(dataset) - train_n
    train_set, test_set = random_split(dataset, [train_n, test_n])

    return DataLoader(train_set, batch_size=32, shuffle=True), \
        DataLoader(test_set, batch_size=32, shuffle=True)

In [10]:
def waveletExtraction(x, wavelet='db1', level=4):
# perform wavelet packet decomposition on signal 1
    wp = pywt.WaveletPacket(x[0], wavelet, mode='symmetric', maxlevel=level)
    coeffs1 = []
    for node in wp.get_level(level, 'natural'):
        if node.path.endswith('a') or node.path.endswith('d'):
            coeffs1.append(node.data)
    coeffs1 = np.concatenate(coeffs1)
    
    # perform wavelet packet decomposition on signal 2
    wp = pywt.WaveletPacket(x[1], wavelet, mode='symmetric', maxlevel=level)
    coeffs2 = []
    for node in wp.get_level(level, 'natural'):
        if node.path.endswith('a') or node.path.endswith('d'):
            coeffs2.append(node.data)
    coeffs2 = np.concatenate(coeffs2)
    
    # concatenate the two coefficient arrays
    feature_vector = np.concatenate((coeffs1, coeffs2))
    
    return feature_vector

In [11]:
dataset, scaler = getDataset()

100%|██████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:35<00:00, 63.39it/s]
C:\Users\rjsou\AppData\Local\Temp\ipykernel_12052\4192844632.py:54: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  self.input = tensor(self.input).type(FloatTensor)


In [12]:
train_loader, test_loader = getSplit(dataset)

In [193]:
class SiameseRNN(nn.Module):
    def __init__(self, input_size=256, hidden_size=512):
        super(SiameseRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=3, batch_first=True)
                
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 6)
        
        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        
        x = x.reshape(batch_size, 1, -1)
        
        #x1 = x[:, 0, :, :].reshape(batch_size, 1, -1)
        #x2 = x[:, 0, :, :].reshape(batch_size, 1, -1)
        
        h0 = torch.zeros(3, batch_size, self.hidden_size).to(DEVICE)
        _, h1 = self.rnn(x, h0)  # Add a batch dimension
        #_, h2 = self.rnn(x2, h0)  # Add a batch dimension
        
        out = h1[-1]
        out = out.reshape(batch_size, -1)
        out = self.fc(out)
        return out

In [194]:
class SiameseLSTM(nn.Module):
    def __init__(self, input_size=256, hidden_size=1024, num_layers=3):
        super(SiameseLSTM, self).__init__()
        
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True)
        
        self.fc = nn.Sequential(
            nn.Linear(2048 , 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 6),
        )

     
    def forward(self, x):
        batch_size = x.shape[0]
        #x1 = x[:, 0, :].unsqueeze(1)
        #x2 = x[:, 0, :].unsqueeze(1)
        x = x.unsqueeze(1)
        
        
        h0 = torch.zeros(self.num_layers*1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers*1, x.size(0), self.hidden_size).to(x.device)
        
        # Forward pass through the LSTM layer
        o, h = self.lstm(x, (h0, c0))
        #out2 = self.lstm(x2, (h0, c0))[1][-1]

        
        #out = torch.cat([out1, out2], dim=0)
        out = h[-1]
        out = out.reshape(batch_size, -1)
        # Pass the last hidden state through the fully connected layer
        out = self.fc(out)
        
        return out

In [195]:
sRNN = SiameseRNN()

In [196]:
sRNN_train_loss, sRNN_val_loss =  train(train_loader, sRNN, val_loader=test_loader, LR=0.001, epochs=400)

Using: cuda:0


Epoch 44/400:  86%|██████████████████████████▌    | 48/56 [00:00<00:00, 129.38batch/s, lastLoss=0.0953, valLoss=0.0937]


KeyboardInterrupt: 

In [ ]:
sLSTM = SiameseLSTM()

In [108]:
sLSTM_train_loss, sLSTM_val_loss =  train(train_loader, sLSTM, val_loader=test_loader, LR=0.001, epochs=400)

Using: cuda:0


Epoch 68/400: 100%|███████████████████████████████| 56/56 [00:00<00:00, 133.49batch/s, lastLoss=0.0846, valLoss=0.0874]


Epoch 136/400: 100%|██████████████████████████████| 56/56 [00:00<00:00, 144.51batch/s, lastLoss=0.0822, valLoss=0.0839]


Epoch 204/400: 100%|██████████████████████████████| 56/56 [00:00<00:00, 214.55batch/s, lastLoss=0.0803, valLoss=0.0899]


Epoch 272/400: 100%|██████████████████████████████| 56/56 [00:00<00:00, 202.89batch/s, lastLoss=0.0777, valLoss=0.0861]


Epoch 340/400: 100%|██████████████████████████████| 56/56 [00:00<00:00, 204.75batch/s, lastLoss=0.0687, valLoss=0.0823]


Epoch 395/400:  79%|████████████████████████▎      | 44/56 [00:00<00:00, 208.41batch/s, lastLoss=0.063, valLoss=0.0767]


KeyboardInterrupt: 